In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

connection_string = os.getenv("VECTOR_DATABASE_URL")

if not connection_string:
    raise ValueError("VECTOR_DATABASE_URL tidak ditemukan di file .env")

API_KEY = os.getenv("OPENROUTER_API_KEY")
BASE_URL = os.getenv("OPENROUTER_BASE_URL")

print("Environment variables loaded.")

Environment variables loaded.


In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_PATH = "../data/permenpan/PERMENPAN NOMOR 38 TAHUN 2017.pdf"

print(f"Memuat file {PDF_PATH}...")
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
splits = text_splitter.split_documents(docs)

print(f"Berhasil! Dokumen dipecah menjadi {len(splits)} potongan (chunks).")

Memuat file ../data/permenpan/PERMENPAN NOMOR 38 TAHUN 2017.pdf...
Berhasil! Dokumen dipecah menjadi 461 potongan (chunks).


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_postgres import PGVector
from langchain_openai import OpenAIEmbeddings

print("Memuat model embedding...")
# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs={'device': 'cpu'},
#     encode_kwargs={'normalize_embeddings': False}
# )

embeddings = OpenAIEmbeddings(
    base_url=BASE_URL,
    api_key=API_KEY, 
    model="qwen/qwen3-embedding-8b"   
) 

COLLECTION_NAME = "permenpan_index" 

vectorstore = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=connection_string,
    use_jsonb=True,
)

print(f"Menyimpan vector ke tabel '{COLLECTION_NAME}' di Postgres...")
vectorstore.add_documents(splits)
print("Penyimpanan selesai!")

Memuat model embedding...
Menyimpan vector ke tabel 'permenpan_index' di Postgres...
Penyimpanan selesai!


In [15]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="x-ai/grok-4.1-fast:free", 
    api_key=API_KEY,
    base_url=BASE_URL,
    temperature=0
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

system_prompt = (
    "Anda adalah asisten AI yang membantu menjawab pertanyaan berdasarkan konteks yang diberikan di bawah ini. "
    "Jika jawaban tidak ada di dalam konteks, katakan 'Saya tidak menemukan informasi tersebut di dokumen'. "
    "Jawablah dengan bahasa Indonesia yang jelas.\n\n"
    "Konteks:\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("RAG Chain siap digunakan!")

RAG Chain siap digunakan!


In [16]:
query = "apa inti utama dokumen ini?"

print(f"Pertanyaan: {query}")
print("-" * 30)

response = rag_chain.invoke({"input": query})

print("JAWABAN AI:")
print(response["answer"])

print("\n" + "-" * 30)
print("Sumber Referensi:")
for i, doc in enumerate(response["context"]):
    print(f"[{i+1}] Halaman {doc.metadata.get('page', '?')}: {doc.page_content[:1000]}...")

Pertanyaan: apa inti utama dokumen ini?
------------------------------
JAWABAN AI:
Inti utama dokumen ini adalah pengaturan penyusunan **Standar Kompetensi Jabatan ASN** yang meliputi **Kompetensi Manajerial**, **Kompetensi Sosial Kultural**, dan persyaratan jabatan. Penyusunan standar tersebut memerlukan **Kamus Kompetensi Teknis**, **Kamus Kompetensi Manajerial** (ditetapkan oleh Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi, Lampiran II), serta **Kamus Kompetensi Sosial Kultural**. Pengkodean standar mengikuti **Lampiran V**. Instansi yang sudah memiliki standar kompetensi dapat menggunakannya sementara hingga standar nasional ditetapkan, dengan penyesuaian maksimal 2 tahun sejak diundangkan (Pasal 17), dan peraturan berlaku sejak diundangkan (Pasal 18).

------------------------------
Sumber Referensi:
[1] Halaman 14: Kompetensi Manajerial, Kompetensi Sosial Kultural, dan
persyaratan jabatan.
2. Penyusunan standar kompetensi jabatan ASN, memerlukan
Kamus Kompetensi 